In [20]:
import sys
sys.path.append('..')
import logging
import rtde.rtde as rtde
import rtde.rtde_config as rtde_config
import time
import math
import subprocess
import os
import ovr
from pyquaternion import Quaternion
import keyboard

ovr.initialize(None)
session, luid = ovr.create()


In [21]:
#logging.basicConfig(level=logging.INFO)

ROBOT_HOST = '192.168.188.123'
ROBOT_PORT = 30004
config_filename = 'pose_pilot_configuration_with_button.xml'

keep_running = True
logging.getLogger().setLevel(logging.INFO)

In [3]:
conf = rtde_config.ConfigFile(config_filename)
state_names, state_types = conf.get_recipe('state')
setp_names, setp_types = conf.get_recipe('setp')
watchdog_names, watchdog_types = conf.get_recipe('watchdog')
button_names, button_types = conf.get_recipe('button')

In [4]:
con = rtde.RTDE(ROBOT_HOST, ROBOT_PORT)
con.connect()

# get controller version
con.get_controller_version()

# setup recipes
con.send_output_setup(state_names, state_types)
setp = con.send_input_setup(setp_names, setp_types)
watchdog = con.send_input_setup(watchdog_names, watchdog_types)
button = con.send_input_setup(button_names, button_types)

INFO:root:Controller version: 3.5.4.0


In [22]:
def quaternion_to_euler_angle(w, x, y, z):
	ysqr = y * y
	
	t0 = +2.0 * (w * x + y * z)
	t1 = +1.0 - 2.0 * (x * x + ysqr)
	X = math.degrees(math.atan2(t0, t1))
	
	t2 = +2.0 * (w * y - z * x)
	t2 = +1.0 if t2 > +1.0 else t2
	t2 = -1.0 if t2 < -1.0 else t2
	Y = math.degrees(math.asin(t2))
	
	t3 = +2.0 * (w * z + x * y)
	t4 = +1.0 - 2.0 * (ysqr + z * z)
	Z = math.degrees(math.atan2(t3, t4))
	
	return X, Y, Z

In [23]:
def quaternion_to_axis_angle(w, x, y, z):
    angle = 2*math.acos(w)
    ax = angle * x / math.sqrt(1-w*w)
    ay = angle * y / math.sqrt(1-w*w)
    az = angle * z / math.sqrt(1-w*w)
    return ax, ay, az
def axis_angle_to_quaternion(ax, ay, az):
    angle = math.sqrt(ax*ax + ay*ay + az*az)
    return math.cos(angle/2), ax*math.sin(angle/2)/angle, ay*math.sin(angle/2)/angle, az*math.sin(angle/2)/angle

In [24]:
def quat_to_neg_quat(q):
    return q[0], -q[1], -q[2], -q[3]

In [25]:
class transform:
    def __init__(self, scale, offset):
        assert len(scale) == len(offset)
        self.scale = scale
        self.offset = offset
    
    def doTransform(self, p):
        return [s*x+b for s, x , b in zip(self.scale, p, self.offset)]

In [26]:
class deltaTransform:
    def __init__(self, original_base, scale, target_base):
        self.o_base = original_base
        self.scale = scale
        self.t_base = target_base
        
    def doTransform(self, original_point):
        # scale and original point delta
        scaled_point = [s*(o-b) for s, b , o in zip(self.scale, self.o_base, original_point)]
        # transform cartesians
        # TODO: make it general, this only works to swap y and z
        scaled_point[1], scaled_point[2] = -scaled_point[2], scaled_point[1]
        
        t_point = [t+s for t, s in zip(self.t_base, scaled_point)]
        return t_point

In [27]:
def quaternion_multiply(quaternion1, quaternion0):
    # Q1*Q0
    w0, x0, y0, z0 = quaternion0
    w1, x1, y1, z1 = quaternion1
    return [-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0]


In [28]:
def setp_to_list(setp):
    list = []
    for i in range(0,6):
        list.append(setp.__dict__["input_double_register_%i" % i])
    return list

def list_to_setp(setp, list):
    for i in range (0,6):
        setp.__dict__["input_double_register_%i" % i] = list[i]
    return setp

def button_to_list(button):
    list = []
    for i in range(0,2):
        list.append(button.__dict__["input_int_register_%i" % (i+1)])
    return list

def list_to_button(button, list):
    for i in range (0,2):
        button.__dict__["input_int_register_%i" % (i+1)] = list[i]
    return button

In [12]:
setp.input_double_register_0 = 0
setp.input_double_register_1 = 0
setp.input_double_register_2 = 0
setp.input_double_register_3 = 0
setp.input_double_register_4 = 0
setp.input_double_register_5 = 0
  
# The function "rtde_set_watchdog" in the "rtde_control_loop.urp" creates a 1 Hz watchdog
watchdog.input_int_register_0 = 0

# button
button.input_int_register_1 = 0
button.input_int_register_2 = 0

T = 5 #sec, Time constant
start_time = time.time()

robot_base = [0.13, -0.61, 0.30, 0, 0, 0] # communication to robot is in meters and radians
controller_base = [-1.10719752312, -0.337188512087, 0.171237841249, 0, 0, 0] # communication with controller is in mm and rad

#scale = [0.0005, 0.0005, 0.0005, 1, 1, 1]
#scale = [0, 0, 0, 0, 0, 0]
scale = [1, 1, 1, 1, 1, 1]
#offset = [r-c*s for r, c, s in zip(robot_base, controller_base, scale)]
#offset[3:6] = [0, 0, 0]

#tr = transform(scale, offset)
tr = deltaTransform(controller_base, scale, robot_base)
# [0.127489254, -0.69790714, 0.3223092, 0.6, -3.0, -0.07]

In [ ]:
## Do not execute from here

In [44]:
#finding transforming quaternine
ts  = ovr.getTrackingState(session, ovr.getTimeInSeconds(), True)
Rtouch = ts.HandPoses[1]
a = Rtouch.ThePose.Position
b = Rtouch.ThePose.Orientation

#b_axis = quaternion_to_axis_angle(b[0],b[1],b[2],b[3])
trans_co_to_rob_cart_quat = (math.cos(math.pi/4),math.sin(math.pi/4), 0, 0)
rob_rot_base = axis_angle_to_quaternion(1.57, 0, 0)

#temp_quat = quaternion_multiply(trans_co_to_rob_cart_quat, [0, b_axis[0],b_axis[1],b_axis[2]])
#temp_quat = quaternion_multiply(temp_quat, quat_to_neg_quat(trans_co_to_rob_cart_quat))
#temp_quat = axis_angle_to_quaternion(temp_quat[1], temp_quat[2], temp_quat[3])
temp_quat = ([b[0],-b[3],b[1],b[2]])
#print([b[0],b[1],-b[3],b[2]])

trans_quat = quaternion_multiply(quat_to_neg_quat(temp_quat), rob_rot_base)
trans_quat
print(a)

ovr.Vector3f(-0.486381053925, -0.408494979143, -0.577978432178)


In [40]:
q1 = [-0.00595320574939, -0.276862770319, 0.0189044270664, -0.960705041885]
q2 = [-0.532132387161, -0.309791833162, 0.254735380411, -0.745636701584]
q3 = [0.71, -0.71, 0, 0]
r = quaternion_multiply(q2, q1)
r = quaternion_multiply(r, q3)
quaternion_to_axis_angle(r[0], r[1], r[2], r[3])
quat_to_neg_quat(q3)

(0.71, 0.71, 0, 0)

In [31]:
#checking
ts  = ovr.getTrackingState(session, ovr.getTimeInSeconds(), True)
Rtouch = ts.HandPoses[1]
a = Rtouch.ThePose.Position
b = Rtouch.ThePose.Orientation
q2 = b
print(q2)


ovr.Quatf(-0.00199410249479, -0.286968052387, 0.950882434845, -0.116051614285)


In [32]:
q1 = [0.0112525718287, -0.264564216137, 0.108742192388, -0.958151519299]
#q1 = [rob_rot_base[0], rob_rot_base[1], rob_rot_base[2], rob_rot_base[3]]
#q2 =[rob_orient[0], rob_orient[1], rob_orient[2], rob_orient[3]]
ww = quaternion_multiply(q2,quat_to_neg_quat(q1))
tt = quaternion_to_axis_angle(ww[0],ww[1],ww[2],ww[3])
print(tt)

(2.3863068953111464, 0.6805755968630406, -0.5963158817627583)


In [33]:
ts  = ovr.getTrackingState(session, ovr.getTimeInSeconds(), True)
Rtouch = ts.HandPoses[1]
a = Rtouch.ThePose.Position
b = Rtouch.ThePose.Orientation

#b_axis = quaternion_to_axis_angle(b[0],b[1],b[2],b[3])

#temp_quat = quaternion_multiply(trans_co_to_rob_cart_quat, [0, b_axis[0],b_axis[1],b_axis[2]])
#temp_quat = quaternion_multiply(temp_quat, quat_to_neg_quat(trans_co_to_rob_cart_quat))
#temp_quat = axis_angle_to_quaternion(temp_quat[1], temp_quat[2], temp_quat[3])
temp_quat = ([b[0],-b[3],b[1],b[2]])

rob_orient = quaternion_multiply(temp_quat, trans_quat)
quaternion_to_axis_angle(rob_orient[0], rob_orient[1], rob_orient[2], rob_orient[3])

(1.570000024655533, 2.2396536300822985e-17, 9.151272897110466e-18)

In [34]:
ts  = ovr.getTrackingState(session, ovr.getTimeInSeconds(), True)
Rtouch = ts.HandPoses[1]
b = Rtouch.ThePose.Orientation
cont_cur_quat = ([b[0],-b[3],b[1],b[2]])

rob_cur_quat = quaternion_multiply(cont_cur_quat, trans_quat)
rob_cur_quat[2], rob_cur_quat[3] = - rob_cur_quat[2], - rob_cur_quat[3] 
rob_cur_quat = quaternion_multiply(rob_cur_quat, rob_base_quat)
quaternion_to_axis_angle(rob_cur_quat[0], rob_cur_quat[1], rob_cur_quat[2], rob_cur_quat[3])

(3.1400000678223625, -3.15405248800973e-17, 1.3227435617184221e-17)

In [35]:
##### Do not execute to here

In [36]:
#finding transforming quaternine
ts  = ovr.getTrackingState(session, ovr.getTimeInSeconds(), True)
Rtouch = ts.HandPoses[1]
b = Rtouch.ThePose.Orientation

rob_base_quat = axis_angle_to_quaternion(1.57, 0, 0)
cont_base_quat = ([b[0],-b[3],b[1],b[2]])
trans_quat = quat_to_neg_quat(cont_base_quat)
#trans_quat = quaternion_multiply(quat_to_neg_quat(cont_base_quat), rob_base_quat)
trans_quat

(-0.00199410249479115,
 -0.11605161428451538,
 0.28696805238723755,
 -0.9508824348449707)

In [16]:
#start data synchronization
if not con.send_start():
    sys.exit()

# control loop
while keep_running:
    # receive the current state
    state = con.receive()
    
    if state is None:
        break;
    
    # Calc new setp
    ts  = ovr.getTrackingState(session, ovr.getTimeInSeconds(), True)
    Rtouch = ts.HandPoses[1]
    a = Rtouch.ThePose.Position
    b = Rtouch.ThePose.Orientation
    
    cont_cur_quat = ([b[0],-b[3],b[1],b[2]])
    rob_rel_quat = quaternion_multiply(cont_cur_quat, trans_quat)
    rob_rel_quat[2], rob_rel_quat[3] = - rob_rel_quat[2], - rob_rel_quat[3] 
    rob_cur_quat = quaternion_multiply(rob_rel_quat, rob_base_quat)
    rob_cur_quat_axis = quaternion_to_axis_angle(rob_cur_quat[0], rob_cur_quat[1], rob_cur_quat[2], rob_cur_quat[3])
    
    new_setp = [a[0], a[1], a[2]] +list(rob_cur_quat_axis);
    new_setp = tr.doTransform(new_setp)
    list_to_setp(setp, new_setp)
    
    # send new setpoint        
    con.send(setp)
    
    # Get button data
    new_button = [0, 0]
    if keyboard.is_pressed('c'):
            new_button[0] = 1
    if keyboard.is_pressed('o'):
            new_button[1] = 1
    print(new_button)
    list_to_button(button, new_button)
    
    # Send button data
    con.send(button)
    
    # kick watchdog
    con.send(watchdog)

INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:

INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:

INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:

INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:skipping package(2)
INFO:root:

[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]

...

[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]

[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]

[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]

[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]

error: [Errno 10054] An existing connection was forcibly closed by the remote host

In [18]:
con.send_pause()
con.disconnect()
sys.stdout.flush()
ovr.destroy(session)
ovr.shutdown()

error: [Errno 10054] An existing connection was forcibly closed by the remote host

In [19]:
sys.stdout.flush()
ovr.destroy(session)
ovr.shutdown()